In [4]:
import re
import os
import json

In [5]:
PATH = os.path.join("/Users/maksimsadkov/Downloads", "biblio.bib")

In [6]:
def get_file_content(path):
    with open(path) as f:
        return f.readlines()

In [7]:
def clear_content(content):
    while '\n' in content:
        content.remove('\n')

In [8]:
def create_format_text(clear_dict):
    result = ""
    if 'Journal' in clear_dict.keys():
        result =  f"- {clear_dict['Author']} {clear_dict['Title']} // {clear_dict['Journal']} .-{clear_dict['Year']}"
    elif 'Publisher' in clear_dict.keys():
        result =  f"- {clear_dict['Author']} {clear_dict['Title']} // {clear_dict['Publisher']} .-{clear_dict['Year']}"
    if 'Volume' in clear_dict.keys():
        result += f".-{clear_dict['Volume']}"
    if 'Pages' in clear_dict.keys():
        result += f".-{clear_dict['Pages']}"
    elif 'Numpages' in clear_dict.keys():
        result += f".-{clear_dict['Numpages']}"
    return result

In [9]:
def create_dict(content_slice):
    content_slice = "".join(content_slice).replace('\n','').replace("{", "")
    content_slice = content_slice.split("},")
    clear_dict = {}
    for i in content_slice:
        left,right = i.split("=")
        left = left.replace(" ", "")
        right = right.replace("}", "")[1:]
        if left == "Author":
            right = right.replace("and", ",")
        clear_dict[left] = right 
    return clear_dict
    
        

In [10]:
def parse_content(content):
    start_pattern = r"@\w+{\S+"
    pattern = '(.*?)[^\\s]= {([^{}]+)}'
    end_pattern = "}\n"
    start_idx, end_idx = -1, -1
    bib_objects = []
    bib_objects_dict = []
    for idx,line in enumerate(content):
        сnt = re.match(pattern, line)
        if re.match(start_pattern, line):
            start_idx = idx
        if re.match(end_pattern, line):
            end_idx = idx
            if start_idx != -1 and end_idx != -1:
                bib_object = content[start_idx+1:end_idx]
                bib_dict_object = create_dict(bib_object)
                bib_objects_dict.append(bib_dict_object)
                # print(create_format_text(clear_slice))
                bib_objects.append(create_format_text(bib_dict_object))
                start_idx, end_idx = -1, -1
    return bib_objects, bib_objects_dict

In [32]:
def to_json(objects, file_name):
    try:
        with open(file_name, 'w') as f:
            json.dump(objects, f)
        return True
    except Exception:
        return False

In [12]:
lines = get_file_content(PATH)

In [13]:
clear_content(lines)

In [14]:
lines

['% This file was created with JabRef 2.10.\n',
 '% Encoding: UTF8\n',
 '@Article{Alonso,\n',
 '  Title                    = {Complex dynamical states in binary mixture convection with weak negative Soret coupling},\n',
 '  Author                   = {Arantxa Alonso and Oriol Batiste and Alvaro Meseguer,and Isabel Mercader},\n',
 '  Journal                  = {PHYSICAL REVIEW E},\n',
 '  Year                     = {2007},\n',
 '  Pages                    = {026310-1-026310-15},\n',
 '  Volume                   = {75},\n',
 '  Owner                    = {che},\n',
 '  Timestamp                = {2014.09.05}\n',
 '}\n',
 '@Article{anderson4,\n',
 '  Title                    = {Traveling wave convection patterns in an annular cell},\n',
 '  Author                   = { Anderson, K. E. and Behringer, R. P. },\n',
 '  Journal                  = {Physica D},\n',
 '  Year                     = {1991},\n',
 '  Pages                    = {444-449},\n',
 '  Volume                   = {51}\n',
 '

In [15]:
cnt, BIB_OBJECTS_DICT = parse_content(lines)

In [16]:
cnt

['- Arantxa Alonso , Oriol Batiste , Alvaro Meseguer,, Isabel Mercader Complex dynamical states in binary mixture convection with weak negative Soret coupling // PHYSICAL REVIEW E .-2007.-75.-026310-1-026310-15',
 '-  Anderson, K. E. , Behringer, R. P.  Traveling wave convection patterns in an annular cell // Physica D .-1991.-51.-444-449',
 '-  Anderson, K. E. , Behringer, R. P.  Long time scales in traveling wave convection patterns  // Phys. Rev. A. .-1990.-145.-323',
 '-  Ascher, U. M. , Mattheij, R. M. M. , Russell, R.D.  Numerical Solution of Boundary Value Problems for Ordinary DifferentialEquations, Prentice Hall Series in Computational Mathematicsl  // Phys. Rev. Lett .-2009.-102.-100',
 "- Author1 , Author2 , Author3 , Author4 ``This is english article'' // Journal .-2012.-2.-200",
 '-  Barten, W. , L\\"ucke, M. , Kamps, M.  Localizad traveling-wave states in binary-fluid mixtures // Phys. Rev. Lett.  .-1991.-66.-2621-2624',
 '- Barten, W. , Lücke, M. , Hort, W. , Kamps, M.  

In [17]:
BIB_OBJECTS_DICT

[{'Title': 'Complex dynamical states in binary mixture convection with weak negative Soret coupling',
  'Author': 'Arantxa Alonso , Oriol Batiste , Alvaro Meseguer,, Isabel Mercader',
  'Journal': 'PHYSICAL REVIEW E',
  'Year': '2007',
  'Pages': '026310-1-026310-15',
  'Volume': '75',
  'Owner': 'che',
  'Timestamp': '2014.09.05'},
 {'Title': 'Traveling wave convection patterns in an annular cell',
  'Author': ' Anderson, K. E. , Behringer, R. P. ',
  'Journal': 'Physica D',
  'Year': '1991',
  'Pages': '444-449',
  'Volume': '51'},
 {'Title': 'Long time scales in traveling wave convection patterns ',
  'Author': ' Anderson, K. E. , Behringer, R. P. ',
  'Journal': 'Phys. Rev. A.',
  'Year': '1990',
  'Pages': '323',
  'Volume': '145'},
 {'Title': ' Numerical Solution of Boundary Value Problems for Ordinary DifferentialEquations, Prentice Hall Series in Computational Mathematicsl ',
  'Author': ' Ascher, U. M. , Mattheij, R. M. M. , Russell, R.D.',
  'Journal': 'Phys. Rev. Lett',
  'Y

In [18]:
result = to_json(BIB_OBJECTS_DICT, "bib_objects.json")

In [19]:
result

True

In [20]:
# Напишем тесты для основных функций

In [33]:
import unittest

In [40]:
class TestLab1Functions(unittest.TestCase):
    def test_clear_content(self):
        st = ["abvgd", "\n"]
        result = clear_content(st)
        self.assertIsNone(result)
        
    def test_to_json(self):
        obj = [1,2,3]
        result = to_json(obj, "file.json")
        self.assertTrue(result)
        file_name = "abvgd.json"
        result = to_json(obj, "")
        self.assertFalse(result)

    def test_parse_content(self):
        value = ['@Article{Alonso,\n',
 '  Title                    = {Complex dynamical states in binary mixture convection with weak negative Soret coupling},\n',
 '  Author                   = {Arantxa Alonso and Oriol Batiste and Alvaro Meseguer,and Isabel Mercader},\n',
 '  Journal                  = {PHYSICAL REVIEW E},\n',
 '  Year                     = {2007},\n',
 '  Pages                    = {026310-1-026310-15},\n',
 '  Volume                   = {75},\n',
 '  Owner                    = {che},\n',
 '  Timestamp                = {2014.09.05}\n',
 '}\n']
        cnt, bib = parse_content(value)
        self.assertEqual(cnt, ['- Arantxa Alonso , Oriol Batiste , Alvaro Meseguer,, Isabel Mercader Complex dynamical states in binary mixture convection with weak negative Soret coupling // PHYSICAL REVIEW E .-2007.-75.-026310-1-026310-15'])
        self.assertEqual(bib, [{'Title': 'Complex dynamical states in binary mixture convection with weak negative Soret coupling',
  'Author': 'Arantxa Alonso , Oriol Batiste , Alvaro Meseguer,, Isabel Mercader',
  'Journal': 'PHYSICAL REVIEW E',
  'Year': '2007',
  'Pages': '026310-1-026310-15',
  'Volume': '75',
  'Owner': 'che',
  'Timestamp': '2014.09.05'}])

In [41]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_clear_content (__main__.TestLab1Functions.test_clear_content) ... ok
test_parse_content (__main__.TestLab1Functions.test_parse_content) ... ok
test_to_json (__main__.TestLab1Functions.test_to_json) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK
